### Notation:
Since objects are considered morphisms (an object $X$ is identified with its identity morphism $\text{id}_X$), we write `X, Y, Z, ...` for objects, `f, g, h, ...` for morphisms that are not objects, and `x, y, z, ...` for morphisms for which itis unknown whether they are objects / identity morphisms.

Objects which are categories are denoted by `C, D, E, ...`.

Functors are denoted by `F, G, H, ...`

In [1]:
class Morphism:
    
    def __init__(self, category, domain, codomain):
        self.category = category
        self.domain = domain
        self.codomain = codomain

In [2]:
class Object(Morphism):

    def __init__(self, category = None):
        # Object() creates the category of categories (whose category is itself)
        if not category:
            category = self
        
        # An object is identified with its identity morphism
        super().__init__(category, self, self)

In [3]:
# Idea is good, but very cumbersome to distinguish between case of Functor and case of just Morphism (same for Object and Category)

# class Functor(Morphism):
    
#     def __init__(self, category, domain, codomain, covariant = True):
#         self.covariant = covariant
#         super().__init__(category, domain, codomain)

In [4]:
# class Category(Object, Functor):

#     def __init__(self, self_category = False):
#         if self_category:
#             super().__init__(self)
#         else:
#             super().__init__(Cat)

In [5]:
class Number(Object):
    
    def __init__(self, n):
        self.n = n
        super().__init__(Cat)

In [6]:
class Representation:
        
    def __init__(self):
        self.ptr = None
        
    def assign(self, ptr):
        self.ptr = ptr
            
class Repr_Symbol(Representation):
    
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        return type(self) == type(other) and self.name == other.name
    
    def dependencies(self):
        if isinstance(self.ptr, Object):
            return [ self.ptr.category ]
        else:
            return [ self.ptr.domain, self.ptr.codomain ]
    
class Repr_Composition(Representation):
    
    def __init__(self, f_list):        
        self.f_list = f_list
    
    def __eq__(self, other):
        return type(self) == type(other) and self.f_list == other.f_list
    
    def dependencies(self):
        return self.f_list

class Repr_Functor(Representation):
    
    def __init__(self, F, x):
        self.F = F
        self.x = x
        
    def __eq__(self, other):
        return type(self) == type(other) and self.F == other.F and self.x == other.x

    def dependencies(self):
        return [ self.F, self.x ]
    
class Repr_Property(Representation):
    
    def __init__(self, prop, data):
        self.prop = prop
        self.data = data
        
    def __eq__(self, other):
        return type(self) == type(other) and self.prop == other.prop and self.data == other.data
    
    def dependencies(self):
        return self.data
    
class Repr_Equality(Representation):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return type(self) == type(other) and ((self.x, self.y) == (other.x, other.y) or (self.x, self.y) == (other.y, other.x))
    
    def dependencies(self):
        return [ self.x, self.y ]

# class Repr_And(Representation):
    
#     def __init__(self, P, Q):
#         self.P = P
#         self.Q = Q
        
#     def __eq__(self, other):
#         return self.P == other.P and self.Q == other.Q
        
#     def depends_on(self, x):
#         return x == self.P or x == self.Q or super().depends_on(x)
        
# class Repr_Or(Representation):
    
#     def __init__(self, P, Q):
#         self.P = P
#         self.Q = Q
        
#     def __eq__(self, other):
#         return self.P == other.P and self.Q == other.Q
    
#     def depends_on(self, x):
#         return x == self.P or x == self.Q or super().depends_on(x)
        
# class Repr_Implies(Representation):
    
#     def __init__(self, P, Q):
#         self.P = P
#         self.Q = Q
        
#     def __eq__(self, other):
#         return self.P == other.P and self.Q == other.Q
    
#     def depends_on(self, x):
#         return x == self.P or x == self.Q or super().depends_on(x)
        
# class Repr_Not(Representation):
    
#     def __init__(self, P):
#         self.P = P
        
#     def __eq__(self, other):
#         return self.P == other.P
        
#     def depends_on(self, x):
#         return x == self.P or super().depends_on(x)


In [7]:
class Diagram:
    
    def __init__(self, parent = None):
        self.parent = parent
        self.data = []
        self.conditions = []
        
        self.morphisms = []
        self.representations = []
        
    def add_data(self, X):
        self.data.append(X)
            
    def add_condition(self, C):
        # TODO: check if condition depends only on the data
        self.conditions.append(C)
    
    def add_morphism(self, x):
        self.morphisms.append(x)
    
    def add_representation(self, rep):
        self.representations.append(rep)
    
    def find_representation(self, rep):
        for r in self.representations:
            if r == rep:
                return r.ptr
                
        return None
    
    def knows(self, x):
        if x in self.morphisms:
            return True
        
        if self.parent:
            return self.parent.knows(x)
        
        return False
    
    def create_name_like(self, name):
        used_names = [ r.name for r in self.representations if isinstance(r, Repr_Symbol) and r.name.startswith(name) ]
        
        while True:
            if name not in used_names:
                return name
            name = name + '\''
            
    def clean(self):
        # Remove all representations and morphisms that are unnecessary (whatever that means)
        # TODO: should this not be a method of Theorem?
        pass
    
    
    # --- Non-essential methods ---
    
    def str_x(self, x):
        for r in self.representations:
            if r.ptr == x:
                if isinstance(r, Repr_Symbol):
                    return r.name
                if isinstance(r, Repr_Functor):
                    return '{}({})'.format(self.str_x(r.F), self.str_x(r.x))
                if isinstance(r, Repr_Property):
                    return '{}({})'.format(r.prop.name, ', '.join([ self.str_x(y) for y in r.data ]))
                if isinstance(r, Repr_Composition):
                    return '.'.join([ self.str_x(f) for f in r.f_list ])
                if isinstance(r, Repr_Equality):
                    return '{} = {}'.format(self.str_x(r.x), self.str_x(r.y))

        if self.parent:
            return self.parent.str_x(x)
        
        return '?'
    
    def __str__(self):
        return '{{ {} }}'.format(', '.join(self.str_x(x) for x in self.morphisms))

In [8]:
class Property:
    
    def __init__(self, name, diagram):
        self.name = name
        self.diagram = diagram

In [9]:
class Theorem:
    
    def __init__(self, name, diagram):
        self.name = name
        self.diagram = diagram
        self.conclusions = []

    def add_conclusion(self, C):
        self.conclusions.append(C)
    
    def try_apply(self, other_diagram, mapping = {}):        
        for x in self.diagram.data:
            if x in mapping:
                continue
            
            x_is_object = isinstance(x, Object)
            for y in other_diagram.morphisms: # TODO: also look in other_diagram.parent ?
                # Categories must match
                if y.category != x.category or (x_is_object and not isinstance(y, Object)):
                    continue
                # (Co)domain must match
                if x.domain in mapping and mapping[x.domain] != y.domain:
                    continue
                if x.codomain in mapping and mapping[x.codomain] != y.codomain:
                    continue
                
                # Try mapping
                mapping[x] = y
                if self.try_apply(other_diagram, mapping):
                    return True
                # If unsuccessful, undo
                del mapping[x]

            return False
        
        # All the data is mapped. Now naturally extend the mapping to all morphisms of self.diagram that depend on the data
        mapping_ext = mapping.copy()
        for x in self.diagram.morphisms:
            if other_diagram.knows(x):
                mapping_ext[x] = x
        
        r_todo = [ r for r in self.diagram.representations if r.ptr not in mapping_ext and not isinstance(r, Repr_Symbol) ]
            
        factory = Factory(other_diagram)
        updates = True
        while updates:
            updates = False
            r_done = []
            for r in r_todo:
                # Can only extend the mapping to this representation if all its dependencies are already mapped
                if any(x not in mapping_ext and x in self.diagram.morphisms for x in r.dependencies()):
                    continue
                                    
                y = factory.from_placeholders(r, mapping_ext)
                mapping_ext[r.ptr] = y
                r_done.append(r)
                updates = True
                
            r_todo = [ r for r in r_todo if r not in r_done ]
                        
        # TODO: check if everything is well-mapped!
            
        # Verify conditions
        for C in self.diagram.conditions:
            C_mapped = mapping_ext[C]
            for x in other_diagram.morphisms:
                if x.category == C_mapped:
                    break
            else:
                return False
        
        # At this point we know we can apply the theorem! We fix the mapping
        mapping.update(mapping_ext)
        
        # Introduce new symbols that come with the theorem
        r_todo = [ r for r in self.diagram.representations if r.ptr not in mapping ]
        while r_todo:
            r_done = []
            for r in r_todo:
                if any(x not in mapping and x in self.diagram.morphisms for x in r.dependencies()):
                    print('Cannot create placeholder for {}. It depends on {}'.format(self.diagram.str_x(r.ptr), ', '.join([ self.diagram.str_x(x) for x in r.dependencies() ])))
                    continue
                    
                y = factory.from_placeholders(r, mapping)
                mapping[r.ptr] = y
                r_done.append(r)
            
            r_todo = [ r for r in r_todo if r not in r_done ]
                    
        # Apply conclusions    
        for C in self.conclusions:
            factory.create_object(other_diagram.create_name_like('c'), mapping[C])
        
        return True                
    
    # --- Non-essential ---
    
    def __str__(self):
        return self.name
    

In [10]:
class Factory:
    
    def __init__(self, diagram):
        self.diagram = diagram
    
    def create_object(self, name, C):
        # C must be a category
        if C.category != Cat:
            raise Exception('That is not a category!')
        
        # Create representation
        rep = Repr_Symbol(name)
        
        # If the representation already exists in the diagram, raise an exception
        if self.diagram.find_representation(rep):
            raise Exception('Name \'{}\' is already used!'.format(name))
        
        # Construct new object
        X = Object(C)
        if C == Cat:
            X.covariant = True
        
        # Assign to representation, and add to the diagram
        rep.assign(X)
        self.diagram.add_morphism(X)
        self.diagram.add_representation(rep)
        
        return X
    
    def create_morphism(self, name, X, Y, **kwargs):
        # The categories of X and Y must be equal
        if X.category != Y.category:
            raise Exception('Cannot construct a morphism between objects of different categories!')

        # Create representation
        rep = Repr_Symbol(name)
        
        # If the representation already exists in the diagram, raise an exception
        if self.diagram.find_representation(rep):
            raise Exception('Name \'{}\' is already used!'.format(name))
        
        # Construct new morphism
        f = Morphism(X.category, X, Y)
        if X.category == Cat:
            f.covariant = kwargs['covariant'] if 'covariant' in kwargs else True
        
        # Assign to representation, and add to the diagram
        rep.assign(f)
        self.diagram.add_morphism(f)
        self.diagram.add_representation(rep)
        
        return f
        
    def create_composition(self, f_list):
        # There must be at least one morphism
        if not f_list:
            raise Exception('Composition requires morphisms!')

        # Obtain domain / codomain
        X, Y = f_list[-1].domain, f_list[0].codomain
        
        # All morphisms must connect
        n = len(f_list)
        for i in range(n - 1):
            if f_list[i].domain != f_list[i + 1].codomain:
                raise Exception('Morphisms do not connect well!')
            
        # Remove all identity morphisms from the list
        f_list = [ f for f in f_list if not isinstance(f, Object) ]
                
        # If the list is empty now, then the result would have been id(X) = id(Y)
        n = len(f_list)
        if n == 0:
            return X
        
        # If there is only one morphism to compose, just return that morphism
        if n == 1:
            return f_list[0]
        
        # Create representation
        rep = Repr_Composition(f_list)
        
        # Check if the representation already exists in the diagram, and if so, return the morphism it points to
        g = self.diagram.find_representation(rep)
        if g:
            return g

        # Construct new object/morphism
        g = Morphism(X.category, X, Y)
        
        # If we are talking about functors, determine whether the resulting functor is covariant or contravariant
        if X.category == Cat:
            g.covariant = True
            for f in f_list:
                g.covariant = (g.covariant ^ f.covariant)
        
        # Assign, and add morphism and representation to the diagram
        rep.assign(g)
        self.diagram.add_morphism(g)
        self.diagram.add_representation(rep)
        
        return g
    
    def apply_functor(self, F, x):
        # x must belong to the domain category of the functor
        if(x.category != F.domain):
            raise Exception('Object/morphism does not belong to functor domain!')
        
        # Create representation
        rep = Repr_Functor(F, x)
        
        # Check if the representation already exists in the diagram, and if so, return the morphism it points to
        F_x = self.diagram.find_representation(rep)
        if F_x:
            return F_x
        
        # Construct new object/morphism
        if isinstance(x, Object):        
            F_x = Object(F.codomain)
        else:
            X = x.domain
            Y = x.codomain
            F_X = self.apply_functor(F, X)
            F_Y = self.apply_functor(F, Y)
            if F.covariant:
                F_x = Morphism(F.codomain, F_X, F_Y)
            else:
                F_x = Morphism(F.codomain, F_Y, F_X)
        
        # Assign, and add morphism and representation to the diagram
        rep.assign(F_x)
        self.diagram.add_morphism(F_x)
        self.diagram.add_representation(rep)
        
        return F_x
    
    def apply_property(self, prop, data):
        # TODO: Check if the data satisfies the diagram of the property. If not, return Null / None / False, or something like that

        # Create representation
        rep = Repr_Property(prop, data)

        # Check if this representation already exists somewhere!
        C = self.diagram.find_representation(rep)
        if C:
            return C        
        
        # Construct new category
        C = Object(Cat)
    
        # Assign, and add object and representation to the diagram
        rep.assign(C)
        self.diagram.add_morphism(C)
        self.diagram.add_representation(rep)
    
        return C

    def create_equality(self, x, y):
        # x and y must lie in the same category
        if x.category != y.category:
            raise Exception('Objects or morphisms can only be equal if in the same category!')
            
        # Create representation
        rep = Repr_Equality(x, y)
        
        # Check if this representation already exists somewhere!
        C = self.diagram.find_representation(rep)
        if C:
            return C
        
        # Construct new category
        C = Object(Cat)
    
        # Assign, and add object and representation to the diagram
        rep.assign(C)
        self.diagram.add_morphism(C)
        self.diagram.add_representation(rep)
    
        return C
    
    def from_placeholders(self, r, M):
        if isinstance(r, Repr_Symbol):
            C = r.ptr.category
            if isinstance(r.ptr, Object):
                return self.create_object(self.diagram.create_name_like(r.name), M[C] if C in M else C)
            else:
                X, Y = r.ptr.domain, r.ptr.codomain
                return self.create_morphism(self.diagram.create_name_like(r.name), M[X] if X in M else X, M[Y] if Y in M else Y)
        
        if isinstance(r, Repr_Composition):
            return self.create_composition([ (M[f] if f in M else f) for f in r.f_list ])

        if isinstance(r, Repr_Functor):
            return self.apply_functor(M[r.F] if r.F in M else r.F, M[r.x] if r.x in M else r.x)

        if isinstance(r, Repr_Property):
            return self.apply_property(r.prop, [ (M[x] if x in M else x) for x in r.data ])
        
        if isinstance(r, Repr_Equality):
            return self.create_equality(M[r.x] if r.x in M else r.x, M[r.y] if r.y in M else r.y)
        
        print('UNSUPPORTED!!!')
        return None
        
        
#     def create_and(self, C_list):
#         # All C must be categories
#         if any(C.category != Cat for C in C_list):
#             raise Exception('The &-operator only applies to categories!')
            
#         # Create representation
#         rep = Repr_And(P, Q)

#         # Check if this representation already exists somewhere!
#         C = self.diagram.find_representation(rep)
#         if C:
#             return C
        
#         # Construct new category
#         C = Object(Cat)
    
#         # Assign, and add object and representation to the diagram
#         rep.assign(C)
#         self.diagram.add_morphism(C)
#         self.diagram.add_representation(rep)
    
#         return C
        

### Core objects

In [11]:
Cat = Object()

_0 = Number(0)
_1 = Number(1)

### Examples

In [12]:
global_diagram = Diagram()
global_diagram.add_morphism(Cat)
global_diagram.add_morphism(_0)
global_diagram.add_morphism(_1)

global_factory = Factory(global_diagram)

# Define in global diagram
Ring = global_factory.create_object('Ring', Cat) # Ring : Cat
Scheme = global_factory.create_object('Scheme', Cat) # Scheme : Cat

Spec = global_factory.create_morphism('Spec', Ring, Scheme, covariant = False) # Spec : Ring ~> Scheme
Mod = global_factory.create_morphism('Mod', Ring, Cat, covariant = False) # Mod : Ring ~> Cat

R = global_factory.create_object('R', Ring) # R : Ring
S = global_factory.create_object('S', Ring) # S : Ring
h = global_factory.create_morphism('h', R, S) # h : R -> S
R_Mod = global_factory.apply_functor(Mod, R) # Mod(R)

# Property affine : diagram (Actually, lets say one needs to define the diagram after saying what the properties name is? That prevents someone from altering a diagram after it is being used. Think about the syntax in that case.)
diagram = Diagram(global_diagram)
factory = Factory(diagram)
X = factory.create_object('X', Scheme)
affine = Property('affine', diagram)

diagram = Diagram(global_diagram)
factory = Factory(diagram)
X = factory.create_object('X', Scheme)
qc = Property('quasi-compact', diagram)

diagram = Diagram(global_diagram)
factory = Factory(diagram)

X = factory.create_object('X', Scheme) # X : Scheme
Y = factory.create_object('Y', Scheme) # Y : Scheme
f = factory.create_morphism('f', X, Y) # f : X -> Y

X_is_affine = factory.apply_property(affine, [ X ]) # affine(X)


In [13]:
str(diagram)

'{ X, Y, f, affine(X) }'

In [14]:
Spec_h = global_factory.apply_functor(Spec, h)

In [15]:
print(global_diagram.str_x(h.domain))
print(global_diagram.str_x(h.codomain))
print(global_diagram.str_x(Spec_h.domain))
print(global_diagram.str_x(Spec_h.codomain))

R
S
Spec(S)
Spec(R)


In [16]:
diagram = Diagram(global_diagram)
factory = Factory(diagram)

A = factory.create_object('A', Ring)
B = factory.create_object('B', Ring)
C = factory.create_object('C', Ring)
f = factory.create_morphism('f', A, B)
g = factory.create_morphism('g', B, C)
h = factory.create_morphism('h', B, B)

gf = factory.create_composition([ g, h, f ])

Spec_f = factory.apply_functor(Spec, f)

In [17]:
str(diagram)

'{ A, B, C, f, g, h, g.h.f, Spec(A), Spec(B), Spec(f) }'

### Create a theorem (affine implies quasi-compact)

In [18]:
thm_diagram = Diagram(global_diagram)
factory = Factory(thm_diagram)
aff_implies_qc = Theorem('Affine implies quasi-compact', thm_diagram)

X = factory.create_object('X', Scheme)
thm_diagram.add_data(X)
X_is_affine = factory.apply_property(affine, [ X ])
thm_diagram.add_condition(X_is_affine)

X_is_qc = factory.apply_property(qc, [ X ])
aff_implies_qc.add_conclusion(X_is_qc)

### Can we apply a theorem?

In [19]:
diagram = Diagram(global_diagram)
factory = Factory(diagram)

X = factory.create_object('X', Scheme) # X : Scheme
Y = factory.create_object('Y', Scheme) # Y : Scheme
f = factory.create_morphism('f', X, Y) # f : X -> Y

Y_is_affine = factory.apply_property(affine, [ Y ]) # affine(Y)

h = factory.create_object('hyp', Y_is_affine) # assume affine(Y)

In [20]:
mapping = {}
if aff_implies_qc.try_apply(diagram, mapping):
    for entry in mapping.items():
        print('{} --> {}'.format(aff_implies_qc.diagram.str_x(entry[0]), diagram.str_x(entry[1])))

X --> Y
affine(X) --> affine(Y)
quasi-compact(X) --> quasi-compact(Y)


In [21]:
str(diagram)

'{ X, Y, f, affine(Y), hyp, affine(X), quasi-compact(X), quasi-compact(Y), c }'

### Another one

In [22]:
thm_diagram = Diagram(global_diagram)
thm_factory = Factory(thm_diagram)
aff_implies_ex_R = Theorem('Definition affine', thm_diagram)

X = thm_factory.create_object('X', Scheme)
thm_diagram.add_data(X)
X_is_affine = thm_factory.apply_property(affine, [ X ])
thm_diagram.add_condition(X_is_affine)

R = thm_factory.create_object('R', Ring)
Spec_R = thm_factory.apply_functor(Spec, R)
X_is_Spec_R = thm_factory.create_equality(X, Spec_R)
aff_implies_ex_R.add_conclusion(X_is_Spec_R)

In [23]:
mapping = {}
if aff_implies_ex_R.try_apply(diagram, mapping):
    for entry in mapping.items():
        print('{} --> {}'.format(aff_implies_ex_R.diagram.str_x(entry[0]), diagram.str_x(entry[1])))

X --> Y
affine(X) --> affine(Y)
R --> R
Spec(R) --> Spec(R)
X = Spec(R) --> Y = Spec(R)


In [24]:
str(diagram)

"{ X, Y, f, affine(Y), hyp, affine(X), quasi-compact(X), quasi-compact(Y), c, R, Spec(R), Y = Spec(R), c' }"

```theorem Aff_imp_QC {
    let X : Scheme
    with affine(X)
    then qcompact(X)
}```

```theorem Aff_imp_X_Spec_R {
    let X : Scheme
    with affine(X)
    then R : Ring 
    then X = Spec(R)
}```

In [26]:
str(aff_implies_qc)

'Affine implies quasi-compact'